[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PyMPDATA/blob/main/examples/PyMPDATA_examples/Magnuszewski_et_al_2025/table_1.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyMPDATA.git/main?urlpath=lab/tree/examples/PyMPDATA_examples/Magnuszewski_et_al_2025/table_1.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PyMPDATA/blob/main/examples/PyMPDATA_examples/Magnuszewski_et_al_2025/table_1.ipynb)

## Paweł Magnuszewski MSc project

Tamble comparing herein computed UPWIND, MPDATA and Monte-Carlo solutions with data from literature

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PyMPDATA-examples')

In [2]:
import os

import numpy as np
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display

from PyMPDATA_examples.Magnuszewski_et_al_2025.asian_option import AsianArithmetic, Settings
from PyMPDATA_examples.Magnuszewski_et_al_2025.common import OPTIONS
from PyMPDATA_examples.Magnuszewski_et_al_2025.monte_carlo import BSModel, FixedStrikeArithmeticAsianOption
from PyMPDATA_examples.Magnuszewski_et_al_2025 import barraquand_data

pd.options.display.float_format = '{:,.3f}'.format

In [3]:
CI = 'CI' in os.environ

s_min = 50
s_max = 200

mc_n_paths = [1000, 10000] if not CI else [10, 100]
mc_seed = 42
mc_path_points = 1000 if not CI else 10

spot = 100
risk_free_rate = 0.1

In [29]:
def run_numeric_and_mc(params, *, nx, ny, nt, variant):
    settings = Settings(**params, r=risk_free_rate, S_max=s_max, S_min=s_min)
    mc_model = BSModel(T=params['T'],
                       sigma=params['sgma'],
                       r=risk_free_rate,
                       M=mc_path_points,
                       S0=spot,
                       seed=mc_seed)
    simulations = {
        k:AsianArithmetic(settings, nx=101, ny=100, nt=2000, eps=1e-10, variant=variant, options=opt) 
        for k, opt in OPTIONS.items()
    }
    results = {}
    
    for k, simulation in simulations.items():
        # print(params, variant, '...')
        simulation.step(simulation.nt)
        simulation_price = simulation.solver.advectee.get()[:, 0]
        results[k] = np.interp(spot, simulation.S, simulation_price)
        # assert np.isfinite(results[k])
        # assert results[k] < 1e3
        # print('...', results[k])
    for mc_n_path in mc_n_paths:
        arithmetic_option = FixedStrikeArithmeticAsianOption(params['T'], params['K'], variant, mc_model, mc_n_path)
        results[f"MC_{mc_n_path}_{variant}"] = arithmetic_option.price_by_mc()
    return results

In [30]:
discretization_parameters = {
    'put' : {
        (.1,.25, 95):{'nx': 201, 'ny': 400, 'nt': 1500},
        (.1,.25,100):{'nx': 201, 'ny': 400, 'nt': 1500},
        (.1,.25,105):{'nx': 201, 'ny': 400, 'nt': 1500},
        (.1,.50, 95):{'nx': 201, 'ny': 300, 'nt': 1500},
        (.1,.50,100):{'nx': 201, 'ny': 300, 'nt': 1500},
        (.1,.50,105):{'nx': 201, 'ny': 300, 'nt': 1500},
        (.1,1.0, 95):{'nx': 251, 'ny': 250, 'nt': 1500},
        (.1,1.0,100):{'nx': 251, 'ny': 250, 'nt': 1500},
        (.1,1.0,105):{'nx': 251, 'ny': 250, 'nt': 1500},
        (.2,.25, 95):{'nx': 251, 'ny': 250, 'nt': 1000},
        (.2,.25,100):{'nx': 251, 'ny': 250, 'nt': 1000},
        (.2,.25,105):{'nx': 251, 'ny': 250, 'nt': 1000},
        (.2,.50, 95):{'nx': 101, 'ny': 200, 'nt': 1000},
        (.2,.50,100):{'nx': 101, 'ny': 200, 'nt': 1000},
        (.2,.50,105):{'nx': 101, 'ny': 200, 'nt': 1000},
        (.2,1.0, 95):{'nx': 101, 'ny': 200, 'nt': 1500},
        (.2,1.0,100):{'nx': 101, 'ny': 200, 'nt': 1500},
        (.2,1.0,105):{'nx': 101, 'ny': 200, 'nt': 1200},
        (.4,.25, 95):{'nx': 101, 'ny': 110, 'nt': 2200},
        (.4,.25,100):{'nx': 101, 'ny': 200, 'nt': 1000},
        (.4,.25,105):{'nx': 101, 'ny': 200, 'nt': 1500},
        (.4,.50, 95):{'nx': 101, 'ny': 110, 'nt': 1000},
        (.4,.50,100):{'nx': 101, 'ny': 110, 'nt': 1000},
        (.4,.50,105):{'nx': 101, 'ny': 110, 'nt': 1000},
        (.4,1.0, 95):{'nx': 101, 'ny': 100, 'nt': 1800},
        (.4,1.0,100):{'nx': 101, 'ny': 100, 'nt': 1780},
        (.4,1.0,105):{'nx': 101, 'ny': 100, 'nt': 1760}
    },
    'call': {
        (.1,.25, 95):{'nx': 121, 'ny': 500, 'nt': 1500},
        (.1,.25,100):{'nx': 101, 'ny': 500, 'nt': 1500},
        (.1,.25,105):{'nx': 101, 'ny': 500, 'nt': 1500},
        (.1,.50, 95):{'nx': 101, 'ny': 500, 'nt': 1500},
        (.1,.50,100):{'nx':  71, 'ny': 400, 'nt': 1500},
        (.1,.50,105):{'nx': 101, 'ny': 400, 'nt': 2000},
        (.1,1.0, 95):{'nx': 101, 'ny': 500, 'nt': 2500},
        (.1,1.0,100):{'nx': 101, 'ny': 500, 'nt': 2500},
        (.1,1.0,105):{'nx': 171, 'ny': 600, 'nt': 3000},
        (.2,.25, 95):{'nx': 101, 'ny': 500, 'nt': 3000},
        (.2,.25,100):{'nx': 101, 'ny': 500, 'nt': 3000},
        (.2,.25,105):{'nx': 151, 'ny': 800, 'nt': 3000},
        (.2,.50, 95):{'nx': 101, 'ny': 300, 'nt': 3000},
        (.2,.50,100):{'nx': 101, 'ny': 300, 'nt': 3000},
        (.2,.50,105):{'nx': 101, 'ny': 400, 'nt': 3000},
        (.2,1.0, 95):{'nx': 101, 'ny': 200, 'nt': 1500},
        (.2,1.0,100):{'nx': 121, 'ny': 200, 'nt': 4000},
        (.2,1.0,105):{'nx': 101, 'ny': 250, 'nt': 5000},
        (.4,.25, 95):{'nx': 101, 'ny': 110, 'nt': 2200},
        (.4,.25,100):{'nx': 101, 'ny': 200, 'nt': 3000},
        (.4,.25,105):{'nx':  81, 'ny': 200, 'nt': 5000},
        (.4,.50, 95):{'nx': 101, 'ny': 110, 'nt': 1000},
        (.4,.50,100):{'nx': 101, 'ny': 110, 'nt': 1000},
        (.4,.50,105):{'nx':  81, 'ny': 150, 'nt': 5000},
        (.4,1.0, 95):{'nx': 101, 'ny': 102, 'nt': 4000},
        (.4,1.0,100):{'nx': 101, 'ny': 100, 'nt': 1780},
        (.4,1.0,105):{'nx': 101, 'ny': 100, 'nt': 1760}
    }
} if not CI else {
    'put' : {
        (.1,.25, 95):{'nx': 21, 'ny': 40, 'nt': 40},
        (.1,.25,100):{'nx': 21, 'ny': 40, 'nt': 40},
        (.1,.25,105):{'nx': 21, 'ny': 40, 'nt': 40},
    },
    'call': {
        (.1,.25, 95):{'nx': 21, 'ny': 40, 'nt': 40},
        (.1,.25,100):{'nx': 21, 'ny': 40, 'nt': 40},
        (.1,.25,105):{'nx': 21, 'ny': 40, 'nt': 40},
    }
}

In [31]:
barraquand_df = pd.DataFrame(columns=barraquand_data.headers)
for line in barraquand_data.table.strip('\n').split('\n'):
    data_row = line.split(',')
    if len(data_row) > 0:
        barraquand_df.loc[len(barraquand_df)] = data_row
barraquand_df['call_price'] = barraquand_df['call_price'].astype(float)
barraquand_df['put_price'] = barraquand_df['put_price'].astype(float)

In [32]:
def calculate_row(row_idx):
    row_data = barraquand_df.iloc[row_idx].astype(float)
    nx_put,ny_put,nt_put = discretization_parameters['put'][(row_data['sigma'], row_data['T'], row_data['K'])].values()
    nx_call,ny_call,nt_call = discretization_parameters['call'][(row_data['sigma'], row_data['T'], row_data['K'])].values()

    simulation_params = {
        'sgma':row_data['sigma'],
        'T':row_data['T'],
        'K':row_data['K']
    }
    call_price = row_data['call_price']
    put_price = row_data['put_price']
    results_call = run_numeric_and_mc(simulation_params, nx=nx_call, ny=ny_call, nt=nt_call, variant='call')
    results_put = run_numeric_and_mc(simulation_params, nx=nx_put, ny=ny_put, nt=nt_put, variant='put')
    return (
        {k: round(v,3) for k, v in results_call.items()},
        {k: round(v,3) for k, v in results_put.items()}, 
        simulation_params, 
        call_price,
        put_price
    )

In [33]:
results_df = pd.DataFrame(columns=[
    'sigma', 'T', 'K',
    'BP_call',
    'UPWIND_call', 'MPDATA_call',
    f'MC_{mc_n_paths[0]}_call', f'MC_{mc_n_paths[1]}_call',
    'BP_put', 'UPWIND_put',
    'MPDATA_put',
    f'MC_{mc_n_paths[0]}_put', f'MC_{mc_n_paths[1]}_put'
])

assert len(discretization_parameters['put']) == len(discretization_parameters['call'])
progbar = IntProgress(max=len(discretization_parameters['put']))
display(progbar)
for i in reversed(range(len(discretization_parameters['put']))):
    call, put, params, call_bp, put_bp = calculate_row(i)
    new_row = [*params.values(), call_bp, *call.values(), put_bp, *put.values()]
    results_df.loc[len(results_df)] = new_row
    progbar.value += 1
results_df['K'] = results_df['K'].astype(int)
display(results_df)

IntProgress(value=0, max=27)

,sigma,T,K,BP_call,UPWIND_call,MPDATA_call,MC_1000_call,MC_10000_call,BP_put,UPWIND_put,MPDATA_put,MC_1000_put,MC_10000_put
0,0.400,1.000,105,8.989,10.567,9.073,8.922,8.919,8.767,10.195,8.804,9.371,8.781
1,0.400,1.000,100,11.213,12.779,NaN,11.092,11.114,6.465,7.910,6.756,7.018,6.451
2,0.400,1.000,95,13.825,15.318,13.859,13.626,13.716,4.550,5.952,"880,671,502.744",5.027,4.529
3,0.400,0.500,105,5.444,7.466,5.510,5.384,5.388,7.748,9.663,7.752,8.190,7.735
4,0.400,0.500,100,7.650,9.696,7.681,7.561,7.564,5.197,7.164,5.193,5.611,5.155
5,0.400,0.500,95,10.425,12.358,10.413,10.257,10.343,3.215,5.096,3.205,3.550,3.178
6,0.400,0.250,105,3.106,5.635,3.240,3.070,3.077,6.735,9.191,6.751,7.069,6.728
7,0.400,0.250,100,5.218,7.840,5.274,5.167,5.163,3.970,6.542,3.978,4.290,3.938
8,0.400,0.250,95,8.151,10.571,8.096,8.005,8.101,2.025,4.419,2.023,2.251,2.000
9,0.200,1.000,105,4.539,7.062,4.644,4.517,4.507,4.356,6.737,4.376,4.708,4.357


In [ ]:
latex_header = """
\\begin{tabular}{ccr|d{2.3}d{2.3}d{2.3}d{2.3}d{2.3}|d{2.3}d{2.3}d{2.3}d{2.3}d{2.3}}
& & & \\multicolumn{5}{l|}{\\textbf{Call Option}} & \\multicolumn{5}{l}{\\textbf{Put Option}} \\\\
$\\sigma$ & $T$ & $K$ & 
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{\\cite{Barraquand_1996}}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{UPWIND}} & 
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{\\bf MPDATA}} & 
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{MC $N=""" + str(mc_n_paths[0]) + """$}} &
 \\multicolumn{1}{c|}{\\rotatebox[origin=l]{90}{MC $N=""" + str(mc_n_paths[1]) + """$}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{\\cite{Barraquand_1996}}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{UPWIND}} & 
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{\\bf MPDATA}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{MC $N=""" + str(mc_n_paths[0]) + """$}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{MC $N=""" + str(mc_n_paths[1]) + """$}} \\\\
"""

In [ ]:
def dump_row_values(row_idx):
    row = results_df.iloc[row_idx]
    ret = f" & {int(row['K'])} & "
    ret += " & ".join([f"{x:#.3g}" for x in row[results_df.columns[3:]].values])
    ret += " \\\\"
    return ret

In [ ]:
with open("table.tex", 'w', encoding='utf-8') as f:
    f.write(latex_header)
    for group in range(len(results_df) // 3):
        group_start_idx = group * 3
        sigma = results_df.iloc[group_start_idx]['sigma']
        time_to_maturity = int(results_df.iloc[group_start_idx]['T'] * 12)
        for i in range(3):
            if i == 0:
                group_start_line = f"\\midrule\n\\multirow{{3}}{{*}}{{{sigma}}} & \\multirow{{3}}{{*}}{{{time_to_maturity}}}"
            else:
                group_start_line = "&"
            line_to_save = group_start_line + dump_row_values(group_start_idx+i) + "\n"
            f.write(line_to_save)
    f.write("\n\\end{tabular}")